# DeepSeek Agent 实战：小红书爆款文案生成助手

本 Notebook 将指导您如何使用 DeepSeek LLM 构建一个能够生成小红书爆款文案的智能 Agent。我们将从需求拆解开始，逐步定义 Agent 的系统提示词 (System Prompt)、外部工具 (Tools)，并实现其核心的工作流程，最终生成符合小红书平台特点的文案。

## 1. 环境准备与DeepSeek API配置

In [1]:
import os
from openai import OpenAI

# 初始化 DeepSeek 客户端

client = OpenAI(
    api_key='ollama',
    base_url="http://localhost:11434/v1/",  # DeepSeek API 的基地址
)

## 2. 需求拆解与Agent任务规划

#### 用户痛点与核心需求：
*   **效率低下：** 人工创作周期长，难以满足高频发布需求。
*   **创意瓶颈：** 难以持续产出新颖、吸引人的爆款创意。
*   **趋势捕捉难：** 实时流行元素难以快速融入文案。
*   **平台特性把握：** 小红书特有风格（标题、正文、标签、表情）难以精准复制。

#### “爆款”文案的特征：
*   **强吸引力标题：** 制造好奇、痛点共鸣、利益点清晰。
*   **沉浸式正文：** 真实体验分享、细节描述、情感共鸣。
*   **精准且多样标签：** 热门话题、品牌词、产品词、垂直领域词。
*   **生动表情符号：** 增强表达力，提升活泼感。
*   **清晰的行动召唤 (CTA)。**

#### Agent 任务规划：核心工作流
1.  **用户指令接收：** 接收产品信息、主题、风格等。
2.  **信息收集 (Web Search/DB Query)：** 实时搜索行业趋势、热门话题、竞品分析、产品卖点。
3.  **内容构思与初稿生成 (LLM)：** 结合所有信息，撰写标题、正文、标签、表情符号。
4.  **风格与格式优化 (LLM)：** 根据小红书平台特点和指定风格，对文案进行润色和结构调整。
5.  **最终输出：** 呈现完整文案。

## 3. 爆款文案生成逻辑与 Prompt 设计

### 3.1 System Prompt (系统提示词)

System Prompt 是 Agent 的“大脑”和“行为准则”。它定义了 Agent 的角色、目标以及工作方式。我们将采用 `Thought-Action-Observation` (ReAct) 模式来引导 DeepSeek 的推理过程。

In [2]:
SYSTEM_PROMPT = """
你是一个资深的小红书爆款文案专家，擅长结合最新潮流和产品卖点，创作引人入胜、高互动、高转化的笔记文案。

你的任务是根据用户提供的产品和需求，生成包含标题、正文、相关标签和表情符号的完整小红书笔记。

请始终采用'Thought-Action-Observation'模式进行推理和行动。文案风格需活泼、真诚、富有感染力。当完成任务后，请以JSON格式直接输出最终文案，格式如下：
```json
{
  "title": "小红书标题",
  "body": "小红书正文",
  "hashtags": ["#标签1", "#标签2", "#标签3", "#标签4", "#标签5"],
  "emojis": ["✨", "🔥", "💖"]
}
```
在生成文案前，请务必先思考并收集足够的信息。
"""

### 3.2 Tools (工具定义)

Agent 的“双手”由一系列可调用的工具组成。这些工具扩展了 LLM 的能力，使其能够获取实时信息、查询数据库或执行特定操作。在这里，我们定义了三个核心工具：

*   `search_web`: 用于搜索互联网上的实时信息，如最新趋势、用户评价等。
*   `query_product_database`: 用于查询产品数据库，获取产品的详细卖点和特点。**此工具为模拟**。
*   `generate_emoji`: 用于根据文案内容生成恰当的表情符号。**此工具为模拟**。

In [3]:
TOOLS_DEFINITION = [
    {
        "type": "function",
        "function": {
            "name": "search_web",
            "description": "搜索互联网上的实时信息，用于获取最新新闻、流行趋势、用户评价、行业报告等。请确保搜索关键词精确，避免宽泛的查询。",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "要搜索的关键词或问题，例如'最新小红书美妆趋势'或'深海蓝藻保湿面膜 用户评价'"
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "query_product_database",
            "description": "查询内部产品数据库，获取指定产品的详细卖点、成分、适用人群、使用方法等信息。",
            "parameters": {
                "type": "object",
                "properties": {
                    "product_name": {
                        "type": "string",
                        "description": "要查询的产品名称，例如'深海蓝藻保湿面膜'"
                    }
                },
                "required": ["product_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "generate_emoji",
            "description": "根据提供的文本内容，生成一组适合小红书风格的表情符号。",
            "parameters": {
                "type": "object",
                "properties": {
                    "context": {
                        "type": "string",
                        "description": "文案的关键内容或情感，例如'惊喜效果'、'补水保湿'"
                    }
                },
                "required": ["context"]
            }
        }
    }
]

### 3.3 模拟工具实现

由于我们无法直接调用真实的外部 API (如Google Search或内部产品数据库)，我们将创建一些模拟 (Mock) 工具函数来演示 Agent 的工作流程。在实际应用中，您需要将这些模拟函数替换为真实的 API 调用。

In [4]:
import random # 用于模拟生成表情
import time # 用于模拟网络延迟

def mock_search_web(query: str) -> str:
    """模拟网页搜索工具，返回预设的搜索结果。"""
    print(f"[Tool Call] 模拟搜索网页：{query}")
    time.sleep(1) # 模拟网络延迟
    if "小红书美妆趋势" in query:
        return "近期小红书美妆流行'多巴胺穿搭'、'早C晚A'护肤理念、'伪素颜'妆容，热门关键词有#氛围感、#抗老、#屏障修复。"
    elif "保湿面膜" in query:
        return "小红书保湿面膜热门话题：沙漠干皮救星、熬夜急救面膜、水光肌养成。用户痛点：卡粉、泛红、紧绷感。"
    elif "深海蓝藻保湿面膜" in query:
        return "关于深海蓝藻保湿面膜的用户评价：普遍反馈补水效果好，吸收快，对敏感肌友好。有用户提到价格略高，但效果值得。"
    else:
        return f"未找到关于 '{query}' 的特定信息，但市场反馈通常关注产品成分、功效和用户体验。"

def mock_query_product_database(product_name: str) -> str:
    """模拟查询产品数据库，返回预设的产品信息。"""
    print(f"[Tool Call] 模拟查询产品数据库：{product_name}")
    time.sleep(0.5) # 模拟数据库查询延迟
    if "深海蓝藻保湿面膜" in product_name:
        return "深海蓝藻保湿面膜：核心成分为深海蓝藻提取物，富含多糖和氨基酸，能深层补水、修护肌肤屏障、舒缓敏感泛红。质地清爽不粘腻，适合所有肤质，尤其适合干燥、敏感肌。规格：25ml*5片。"
    elif "美白精华" in product_name:
        return "美白精华：核心成分是烟酰胺和VC衍生物，主要功效是提亮肤色、淡化痘印、改善暗沉。质地轻薄易吸收，适合需要均匀肤色的人群。"
    else:
        return f"产品数据库中未找到关于 '{product_name}' 的详细信息。"

def mock_generate_emoji(context: str) -> list:
    """模拟生成表情符号，根据上下文提供常用表情。"""
    print(f"[Tool Call] 模拟生成表情符号，上下文：{context}")
    time.sleep(0.2) # 模拟生成延迟
    if "补水" in context or "水润" in context or "保湿" in context:
        return ["💦", "💧", "🌊", "✨"]
    elif "惊喜" in context or "哇塞" in context or "爱了" in context:
        return ["💖", "😍", "🤩", "💯"]
    elif "熬夜" in context or "疲惫" in context:
        return ["😭", "😮‍💨", "😴", "💡"]
    elif "好物" in context or "推荐" in context:
        return ["✅", "👍", "⭐", "🛍️"]
    else:
        return random.sample(["✨", "🔥", "💖", "💯", "🎉", "👍", "🤩", "💧", "🌿"], k=min(5, len(context.split())))

# 将模拟工具函数映射到一个字典，方便通过名称调用
available_tools = {
    "search_web": mock_search_web,
    "query_product_database": mock_query_product_database,
    "generate_emoji": mock_generate_emoji,
}

## 4. 实战：构建小红书文案生成 Agent

现在，我们将把 System Prompt、工具定义和模拟工具函数整合起来，构建出能够自动执行的 DeepSeek Agent 工作流。核心是 `generate_rednote` 函数，它通过一个循环来模拟 Agent 的 `Thought-Action-Observation` 过程。

ollama DEEPSEEK-R1 不支持工具调用？

In [28]:
import json
import re

def generate_rednote(product_name: str, tone_style: str = "活泼甜美") -> str:
    """
    使用 DeepSeek Agent 生成小红书爆款文案。
    
    Args:
        product_name (str): 要生成文案的产品名称。
        tone_style (str): 文案的语气和风格，如"活泼甜美"、"知性"、"搞怪"等。
        max_iterations (int): Agent 最大迭代次数，防止无限循环。
        
    Returns:
        str: 生成的爆款文案（JSON 格式字符串）。
    """
    
    print(f"\n🚀 启动小红书文案生成助手，产品：{product_name}，风格：{tone_style}\n")
    
    # 存储对话历史，包括系统提示词和用户请求
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"请为产品「{product_name}」生成一篇小红书爆款文案。要求：语气{tone_style}，包含标题、正文、至少5个相关标签和5个表情符号。请以完整的JSON格式输出，并确保JSON内容用markdown代码块包裹（例如：```json{{...}}```）。"}
    ]

    # 调用 DeepSeek API，传入对话历史和工具定义
    response = client.chat.completions.create(
        model="deepseek-r1",
        messages=messages
    )
    
    response_message = response.choices[0].message.content
    # response_message = re.sub(r'<think>.*?</think>', '', response.choices[0].message.content, flags=re.DOTALL)
    print(f"[模型生成结果] {response_message}")
                
    # --- START: 添加 JSON 提取和解析逻辑 ---
    response_message = re.sub(r'<think>.*?</think>', '', response_message, flags=re.DOTALL)
    json_string_match = re.search(r"```json\s*(\{.*\})\s*```", response_message, re.DOTALL)
    
    if json_string_match:
        extracted_json_content = json_string_match.group(1)
        try:
            final_response = json.loads(extracted_json_content)
            print("Agent: 任务完成，成功解析最终JSON文案。")
            return json.dumps(final_response, ensure_ascii=False, indent=2)
        except json.JSONDecodeError as e:
            print(f"Agent: 提取到JSON块但解析失败: {e}")
            print(f"尝试解析的字符串:\n{extracted_json_content}")
            messages.append(response_message) # 解析失败，继续对话
    else:
        # 如果没有匹配到 ```json 块，尝试直接解析整个 content
        try:
            final_response = json.loads(response_message)
            print("Agent: 任务完成，直接解析最终JSON文案。")
            return json.dumps(final_response, ensure_ascii=False, indent=2)
        except json.JSONDecodeError:
            print("Agent: 生成了非JSON格式内容或非Markdown JSON块，可能还在思考或出错。")
            messages.append(response_message) # 非JSON格式，继续对话
    # --- END: 添加 JSON 提取和解析逻辑 ---

## 5. 实际测试与文案生成

现在，让我们调用我们构建的 `generate_rednote` 函数，看看它能生成什么样的爆款文案！

In [29]:
# 测试案例 1: 深海蓝藻保湿面膜
product_name_1 = "深海蓝藻保湿面膜"
tone_style_1 = "活泼甜美"
result_1 = generate_rednote(product_name_1, tone_style_1)

print("\n--- 生成的文案 1 ---")
print(result_1)


🚀 启动小红书文案生成助手，产品：深海蓝藻保湿面膜，风格：活泼甜美

[模型生成结果] <think>
首先，用户要求我作为资深的小红书爆款文案专家来创作文案。产品是「深海蓝藻保湿面膜」。需求包括：

- 语气活泼甜美

- 包含标题、正文、至少5个相关标签和5个表情符号

输出格式必须是完整的JSON，并用markdown代码块包裹，例如：`{...}`。

在生成前，我需要使用'Thought-Action-Observation'模式进行推理：

1. **Thought**: 思考阶段。收集足够的信息关于产品、目标受众、潮流趋势等。

2. **Action**: 行动阶段。基于思考创建文案。

3. **Observation**: 观察阶段，但用户没有指定额外的观察部分，所以可能只需专注于生成文案，并在输出后以JSON结束。

重点是生成一个高互动、高转化的笔记文案，风格活泼、真诚、富有感染力。

现在，分析产品「深海蓝藻保湿面膜」：

- 产品名称：深海蓝藻保湿面膜

- 核心卖点：保湿、使用深海蓝藻成分。可能暗示其纯天然、深层滋养、抗氧化等好处。

- 目标受众：小红书用户通常对美容护肤感兴趣，尤其是年轻女性，追求皮肤健康、美丽和保养。

- 营销角度：需要突出产品的独特卖点（USP），如保湿效果强、温和不刺激、适合敏感肌等。强调潮流元素，比如环保、天然成分、高科技提取等，结合当前小红书上的热门话题。

用户指定至少5个相关标签和5个表情符号：

- 相关标签：可能基于产品主题的内容标签，如#护肤 #保湿 #面膜 #深海蓝藻 #美丽秘诀 等。需要确保原创或合适地使用。

- 表情符号：应该活泼甜美，所以选择粉色、微笑相关的emoji，比如💖✨😊 等。至少5个。

文案结构：

- 标题：吸引眼球，活泼甜美风格。或许用一个悬念或情感号召的标题。

- 正文：需要引人入胜，分享个人体验、好处和原因选择它。保持故事性强、真诚感染力高。加入互动元素如提问、邀请评论等来提高互动。

整体JSON格式：

{
  "title": "小红书标题",
  "body": "小红体正文",
  "hashtags": ["#标签1", "#标签2", "#标签3", "#标签4", "#标签5"],
  "emojis": ["✨", "💖", ...] // 列

In [30]:
# 测试案例 2: 美白精华
product_name_2 = "美白精华"
tone_style_2 = "知性温柔"
result_2 = generate_rednote(product_name_2, tone_style_2)

print("\n--- 生成的文案 2 ---")
print(result_2)


🚀 启动小红书文案生成助手，产品：美白精华，风格：知性温柔

[模型生成结果] <think>
首先，用户指定了产品是“美白精华”，并给出了具体的风格要求：语气知性温柔。意思是文案的语气要像一个智慧、温柔的女人在分享她的体验一样。

任务包括生成包含标题、正文、相关标签和表情符号的完整小红书笔记，并以JSON格式输出。  
JSON格式必须严格遵守：

{
  "title": "小红书标题",
  “body” ： ” 小红书正文“，
  "hashtags" : ["#label1", "#label2", ...，至少5个相关标签]，
  "emojis" : ["✨", "🔥", ..." 至少五个表情符号]

}

输出时要用markdown代码块包裹JSON内容，例如：```json { ... }```

接下来是' Thought-Action-Observation'模式：

1. **Thought（思考）**：先思考并收集足够的信息。
   - 产品：美白精华。这是针对皮肤护理的产品，帮助改善肤色不均、暗沉等问题。
   - 风格：知性温柔。意思要柔和、亲切的语气，像在温柔地分享经验一样，而不是夸张或刺激性的广告语。
   – 情感基调：真诚、富有感染力，激发读者共鸣。
   - 流行元素：小红书喜欢真实故事、好物推荐、生活 tips。需要结合美白产品的卖点和常见话题。

2. **Action（行动）**：基于思考生成文案。
   - 标题：应吸引人，简洁，并反映内容。例如，关于美白精华的个人使用体验或好处。
   - 正文：要生动描述使用过程、效果感受。结构上可以是介绍自己的皮肤问题、如何发现这个产品、日常使用方法、效果变化、小提醒等。语气知性温柔：使用柔和措辞，如“我发现”、“它真的帮了我很多”或分享故事的方式。
   - Hashtags（至少5个相关标签）：需要与美白精华主题相关的标签。比如#美白秘籍, #变白不反弹, #护肤心得, #焕肤仪搭配等等。要确保是热门标签，以提高互动性。

3. **Observation（观察）**：在生成后，考虑JSON格式和整体结构。
   - 正文长度：小红书笔记通常简洁但有感染力，避免太长太多段落。
   - 表情符号至少5个：选择与内容匹配的emoji。如✨代表惊喜, 💖表示美, 但我也可以

In [31]:
# 测试案例 3: 环保咖啡杯
product_name_3 = "环保咖啡杯"
tone_style_3 = "俏皮可爱"
result_3 = generate_rednote(product_name_3, tone_style_3)

print("\n--- 生成的文案 3 ---")
print(result_3)


🚀 启动小红书文案生成助手，产品：环保咖啡杯，风格：俏皮可爱

[模型生成结果] <think>
首先，用户的查询是让我为产品「环保咖啡杯」生成一篇小红书爆款文案。要求包括：语气俏皮可爱，包含标题、正文、至少5个相关标签和5个表情符号。

我需要采用'Thought-Action-Observation'模式进行推理和行动：
- Thought: 思考阶段，收集足够的信息。
- Action: 行动阶段，基于思考创作文案。
- Observation: 观察阶段，在输出前确保满足要求。

任务是生成完整的JSON格式的笔记文案，并用markdown代码块包裹它。JSON格式指定为：
{
  "title": "小红书标题",
  "body": "小红"正文",
  "hashtags": ["#标签1", "#标签2", "#标签3", "#标签4", "#标签5"],
  "emojis": ["✨", "🔥", "#标签6"]
}

 hashtags 是相关标签，emojis 是表情符号。要求至少5个相关标签和5个表情符号。

产品是环保咖啡杯。我需要结合它的卖点：环保、可持续性、适合日常生活使用，减少一次性杯子的浪费等。同时，文案要活泼、真诚、富有感染力。

Thought：收集信息
- 产品卖点：
1. 环保材质（如可重复使用）。
2. 节约成本（长期使用比买咖啡便宜）。
3. 时尚设计（可能有各种外观，适合年轻人）。
4. 方便携带，适合咖啡爱好者或外卖族。
5. 减少碳足迹。
- 最新潮流：小红书上流行环保主题、可持续生活方式。目标用户可能是关注环境的年轻女性或咖啡文化的人群。
- 语气要求：俏皮可爱。这意味着用轻松、活泼的语言，带点幽默和亲切感。避免太正式，使用感叹词、俏皮短语。

标题应该吸引人，富有感染力，包含关键词"环保咖啡杯"，并以可爱的方式呈现。

正文需要：
- 引入产品。
- 描述好处：环保、省钱、时尚。
- 分享个人体验或故事，增加真诚感。
- 鼓励互动：呼吁用户点赞、收藏、评论。
- 保持简洁有趣，适合小红书笔记长度。

表情符号（emojis）：至少5个。选择与主题相关、活泼的emoji，如✨表示发光的惊喜，☕表示咖啡，♻️表示回收（环保），🌿表示绿色生活等。

hashtags标签：至少5个。需要是热门的相关标签：
- #环

### 格式化 小红书文案

**格式化函数 `format_rednote_for_markdown` 的功能：**

1. 解析 JSON 字符串。
2. 提取标题、正文、标签和表情符号。
3. 将它们组合成一个易读的 Markdown 格式的文本。


**工作方式：**

1. **解析 JSON**：使用 `json.loads()` 将输入的字符串转换为 Python 字典。如果解析失败，会返回一个错误信息。
2. **提取数据**：使用 `.get()` 方法从字典中安全地提取 `title`、`body` 和 `hashtags`。使用 `.get()` 的好处是，如果某个键不存在，它会返回一个默认值（例如 `None` 或空列表），而不是抛出 `KeyError`。
3. **构建 Markdown 标题**：将 `title` 格式化为 Markdown 的二级标题 (`## Title`)。
4. **处理正文**：直接使用 `body`。由于小红书正文中的换行很重要，我们保留它们。
5. **处理 Hashtags**：将 `hashtags` 列表中的每个标签用空格连接起来，形成一行。
6. **表情符号 (Emojis)**：在小红书的实际发布中，表情符号通常已经嵌入在标题和正文中了。这个函数没有单独列出它们，因为这通常不是最终发布格式的一部分。如果需要，可以取消注释相关代码来单独显示它们。
7. **返回结果**：返回拼接好的 Markdown 字符串，并使用 `.strip()` 去除可能存在于末尾的多余空白。

In [32]:
import json

def format_rednote_for_markdown(json_string: str) -> str:
    """
    将 JSON 格式的小红书文案转换为 Markdown 格式，以便于阅读和发布。

    Args:
        json_string (str): 包含小红书文案的 JSON 字符串。
                           预计格式为 {"title": "...", "body": "...", "hashtags": [...], "emojis": [...]}

    Returns:
        str: 格式化后的 Markdown 文本。
    """
    try:
        data = json.loads(json_string)
    except json.JSONDecodeError as e:
        return f"错误：无法解析 JSON 字符串 - {e}\n原始字符串：\n{json_string}"

    title = data.get("title", "无标题")
    body = data.get("body", "")
    hashtags = data.get("hashtags", [])
    # 表情符号通常已经融入标题和正文中，这里可以选择是否单独列出
    # emojis = data.get("emojis", []) 

    # 构建 Markdown 文本
    markdown_output = f"## {title}\n\n" # 标题使用二级标题
    
    # 正文，保留换行符
    markdown_output += f"{body}\n\n"
    
    # Hashtags
    if hashtags:
        hashtag_string = " ".join(hashtags) # 小红书标签通常是空格分隔
        markdown_output += f"{hashtag_string}\n"
        
    # 如果需要，可以单独列出表情符号，但通常它们已经包含在标题和正文中
    # if emojis:
    #     emoji_string = " ".join(emojis)
    #     markdown_output += f"\n使用的表情：{emoji_string}\n"
        
    return markdown_output.strip() # 去除末尾多余的空白

In [33]:
# --- 示例使用 ---
# 假设这是 generate_rednote 函数的输出
generated_json_output = """
{
  "title": "✨ 28天逆袭冷白皮！这款美白精华让我告别暗沉痘印 🌟",
  "body": "姐妹们！我终于找到了我的本命美白精华！💖\\n\\n作为一个常年熬夜➕痘印困扰的混油皮，肤色暗沉一直是我的心头大患。直到遇见了这款美白精华，简直打开了新世界的大门！🤩\\n\\n🌟 核心成分：烟酰胺+VC衍生物，双管齐下，提亮肤色效果绝绝子！\\n💧 质地轻薄到爆炸，上脸秒吸收，完全不会黏腻，油皮姐妹放心冲！\\n🌿 用了28天，痘印肉眼可见变淡了，整张脸都透亮了起来，素颜也能打！\\n\\n使用方法也很简单：早晚洁面后，滴2-3滴在手心，轻轻按压上脸，后续再叠加保湿产品就OK啦～\\n\\n真心推荐给所有想要均匀肤色、告别暗沉的姐妹！入股不亏！💖",
  "hashtags": ["#美白精华", "#提亮肤色", "#淡化痘印", "#护肤好物", "#冷白皮"],
  "emojis": ["✨", "💖", "🤩", "💧", "🌿"]
}
"""

# 调用格式化函数
markdown_note = format_rednote_for_markdown(generated_json_output)

# 打印结果
print("--- 格式化后的小红书文案 (Markdown) ---")
print(markdown_note)

# --- 另一个例子，假设JSON解析失败 ---
invalid_json_output = "{'title': 'Test', 'body': 'This is not valid json'}" # 使用单引号，非法
markdown_error_note = format_rednote_for_markdown(invalid_json_output)
print("\n--- 格式化错误示例 ---")
print(markdown_error_note)


--- 格式化后的小红书文案 (Markdown) ---
## ✨ 28天逆袭冷白皮！这款美白精华让我告别暗沉痘印 🌟

姐妹们！我终于找到了我的本命美白精华！💖

作为一个常年熬夜➕痘印困扰的混油皮，肤色暗沉一直是我的心头大患。直到遇见了这款美白精华，简直打开了新世界的大门！🤩

🌟 核心成分：烟酰胺+VC衍生物，双管齐下，提亮肤色效果绝绝子！
💧 质地轻薄到爆炸，上脸秒吸收，完全不会黏腻，油皮姐妹放心冲！
🌿 用了28天，痘印肉眼可见变淡了，整张脸都透亮了起来，素颜也能打！

使用方法也很简单：早晚洁面后，滴2-3滴在手心，轻轻按压上脸，后续再叠加保湿产品就OK啦～

真心推荐给所有想要均匀肤色、告别暗沉的姐妹！入股不亏！💖

#美白精华 #提亮肤色 #淡化痘印 #护肤好物 #冷白皮

--- 格式化错误示例 ---
错误：无法解析 JSON 字符串 - Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
原始字符串：
{'title': 'Test', 'body': 'This is not valid json'}


In [34]:
# 调用格式化函数
markdown_note = format_rednote_for_markdown(result_1)

# 打印结果
print("--- 格式化后的小红书文案 (Markdown) ---")
print(markdown_note)

--- 格式化后的小红书文案 (Markdown) ---
## 🌊小仙女必备！深海蓝藻保湿面膜让我肌肤水当歌✨

哇哦～宝贝们！你们知道吗？我最近迷上了一款超级神奇的面膜叫「深海蓝藻保湿面膜」，它真的是从大自然中汲取了宝藏成分！💖每用一次都感觉自己像被大海宠爱的小仙子一样水润动人。先来说说我为什么选它吧～我的皮肤最近总是缺水、干燥起皮，加上换季又有点敏感，但这款面膜用了深海蓝藻提取物超级温和不刺激，能深层补水保湿，还带点抗氧化功效哦！🌟第一次敷上约15分钟就觉得脸蛋子软弹弹的，像喝饱了水一样✨；第二天醒来妆前拍一拍，肌肤光泽度up up，粉底服帖不卡纹～这感觉也太幸福了吧！💕而且它质地轻薄不粘腻，洗完也不会紧绷，简直是打工人的急救必备！💦大家还在为干燥烦恼吗？如果是的话，一定要试试这款面膜，真的会爱上它的水疗感！如果你现在就有保湿的问题，别犹豫啦，赶紧入手一瓶放在包里、办公室抽屉，时刻守护水分肌肤～相信我哦，用了之后你会像我一样惊喜尖叫的！😍评论区告诉我你们用过什么好物吧～我们一起变美！💖#深海蓝藻面膜 #保湿护肤 #美丽秘诀 #小红书推荐 #痘痘肌急救


In [35]:
# 调用格式化函数
markdown_note = format_rednote_for_markdown(result_2)

# 打印结果
print("--- 格式化后的小红书文案 (Markdown) ---")
print(markdown_note)

--- 格式化后的小红书文案 (Markdown) ---
## 我的温柔美白秘密💖

大家好呀！作为一名温和又知性的护肤爱好者，“总觉得每个人的故事里都有值得分享的美丽转折。”这款美白精华真的是我发现的好宝贝～它采用高科技成分，帮助我从内而外提升肤色均匀度。以前总是被暗沉和色斑困扰，“每天早晚轻轻按摩使用的方法”让我皮肤慢慢变得透亮，心情也随之明亮起来。坚持了两个月后，我的整体气色更好了，摸起来的细腻感也超赞！在这里想跟姐妹们亲切提醒：“美白不仅仅是护肤小事哦”，要坚持科学的用量和防晒才行～快来关注我多分享一些日常Tips吧！

#美白精华 #知性护理 #肌肤焕白 #美丽转变 #日常美白


In [36]:
# 调用格式化函数
markdown_note = format_rednote_for_markdown(result_3)

# 打印结果
print("--- 格式化后的小红书文案 (Markdown) ---")
print(markdown_note)

--- 格式化后的小红书文案 (Markdown) ---
## 我的第一个环保咖啡杯！可爱到爆的变身神器💖

Hey宝贝们～你们有没有试过一天喝完咖啡就丢掉纸杯的感觉？那随手一扔，地球在哭泣啊😢！但今天我要安利这个超有趣的环保咖啡杯，它简直就是我的秘密武器呢☕️ 轻便耐用，颜值爆表✨ 颜色多样，选择你的ins风最爱～以前我总是用一次性杯子太丢脸了🌿 现在装得了我的爱喝咖啡，还能节约开支💰（少买那么几壶咖啡省下来的钱诶～），环保党必须拥有它的理由大公开：不仅减少塑料浪费♻️ 而且洗起来超轻松！姐妹们说是不是超级实用？下次约个闺蜜出来喝奶茶吧，我们都带着自己的杯子走~ 呼吁你也来试试，地球会感谢你的选择的哦❤️ 评论区分享你的体验吧～让我们一起变绿能侠！👍

#环保咖啡杯 #可持续生活 #日常必备好物 #ins风家居 #绿色出行


## 6. 评估与优化

文案生成并非一蹴而就，需要持续的评估和优化。本节讨论一些评估方法和优化策略。

#### 评估文案质量：
*   **客观量化评估 (数据)：**
    *   **点赞/收藏/评论/分享：** 基础互动
    *   **曝光/阅读/点击/涨粉：：** 流量与曝光
    *   **停留时长/截图率：** 用户行为。
    *   **商品页浏览/加购/ROI/成交转化：** 商业价值
    *   **爆文率/同类横向对比：** 竞争对比
*   **主观内部评估 (人工)：**
    *   **相关性：** 是否符合产品特点和主题。
    *   **吸引力：** 标题是否抓人，内容是否流畅。
    *   **合规性：** 是否有敏感词、违规宣传。
    *   **风格匹配：** 是否符合小红书调性和指定语气。
    *   **用户画像：** 目标人群年龄、地域、兴趣标签。



#### 优化迭代方法：
*   **Prompt 调整：** 根据评估结果，精修 System Prompt、User Prompt，增加或修改 Few-shot 示例。
*   **工具扩充：** 引入新的工具（如敏感词检测工具、竞品分析工具）。
*   **RAG (检索增强生成)：** 结合更精准的内部知识库，减少幻觉。


## 7. 总结与展望

通过本次实战，我们成功构建了一个基于 DeepSeek Agent 的小红书爆款文案生成助手。我们学习了如何拆解需求、设计 Prompt、定义工具，并实现 Agent 的核心工作流。

Agent 在内容营销领域的潜力巨大，未来可以进一步拓展到：

*   **超个性化内容：** 根据用户数据，生成一对一的定制文案。
*   **多模态内容创作：** 结合图片、视频生成，实现图文音视频一体化。
*   **智能营销决策：** Agent 不仅生成内容，还能分析效果并给出投放建议。
*   **跨平台适配：** 快速生成适应不同社交媒体平台风格的文案。

同时，我们也需关注挑战，如确保内容真实性、处理高度主观情感、与现有工作流的无缝集成等。Agent 技术仍在快速发展，期待未来能带来更多惊喜！